In [4]:
import pandas as pd
import astropy.io.fits as fits
import matplotlib.pyplot as plt

from completeness import create_completeness_dataframe
from mass_function import get_weighted_mass_histogram

In [3]:
gkvInputCatv02_raw = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/GAMA/gkvInputCatv02/gkvInputCatv02.fits')[1].data
SM_cat_raw = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/GAMA/merged/StellarMass-gkvScience/mergedStellarMass-gkvScience')[1].data

In [4]:
REGIONS_OF_SKY = {
    # 'G02': {'RAcen': (30.20, 38.80), 'DECcen': (-10.25, -3.72)},
    'G09': {'RAcen': (129.0, 141.0), 'DECcen': (-2.0, 3.0)},
    'G12': {'RAcen': (174.0, 186.0), 'DECcen': (-3.0, 2.0)},
    'G15': {'RAcen': (211.5, 223.5), 'DECcen': (-2.0, 3.0)},
    'G23': {'RAcen': (339.0, 351.0), 'DECcen': (-35.0, -30.0)},
}

In [ ]:
completeness_all_df = pd.DataFrame()

# Loop over regions and create completeness DataFrames
for region_name, region_params in REGIONS_OF_SKY.items():
    completeness_region_df = create_completeness_dataframe(big_survey=gkvInputCatv02,
                                                           small_survey=completeness_catalog,
                                                           flux_type='flux_rt',
                                                           region=region_name)
    completeness_region_df['region'] = region_name
    completeness_all_df = pd.concat([completeness_all_df, completeness_region_df], ignore_index=True)

In [ ]:
mass_histogram_catalog_dataframe = pd.DataFrame()
mass_histogram_catalog_dataframe['uberID'] = mass_histogram_catalog['uberID'].byteswap().newbyteorder()
mass_histogram_catalog_dataframe['mstar'] = mass_histogram_catalog['mstar'].byteswap().newbyteorder()
mass_histogram_catalog_dataframe['comovingdist'] = mass_histogram_catalog['comovingdist'].byteswap().newbyteorder()


In [ ]:
full_mass_completeness_dataframe = pd.merge(mass_histogram_catalog_dataframe, completeness_all_df,
                            on='uberID', how='left')

In [ ]:
region_colors = {'G09': 'red', 'G12': 'lime', 'G15': 'blue', 'G23': 'cyan'}
fig, ax = plt.subplots(figsize=(24, 16))
for region_name in REGIONS_OF_SKY.keys():
    mass_hist = get_weighted_mass_histogram(input_mass_completeness_dataframe=full_mass_completeness_dataframe, region_name=region_name)
    error = calculate_error(mass_hist, len(mass_hist))      # not correct

    plt.errorbar(
        MASS_BINS[:-1],
        mass_hist,
        yerr=error,
        label=region_name,
        color=region_colors.get(region_name),
    )

plt.plot(x, y, label='Driver Interpolated Line', color='magenta', linewidth=4)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Stellar Mass ($M_{\odot} \ h_{70}^{-2}$)', fontsize=16)
plt.ylabel('Number Density ($\mathrm{Mpc}^{-3} \ dex^{-1} \ h_{70}^{3}$)', fontsize=16)
plt.xlim((10**6.5, 10**12))
plt.ylim((1e-5, 1e0))
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend()
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()
print(calculate_error(mass_hist, len(mass_hist)))
